<details><summary> </summary>

# Skip notebook test

</details>

In [ ]:
import os
import pandas as pd
import notebook_utils.notebook_helpers as utils
from cuopt_thin_client import CuOptServiceClient

# Service Team Routing
## Capacitated Vehicle Routing Problem with Time Windows (CVRPTW)

The ability of service providers to set service time windows allows for easier and more dependable coordination between the service provider and their customers, while increasing overall customer satisfaction.

In this scenario we have a number of service order locations with associated time windows and service times (time on-site to complete service). Each technician has an associated availability, ability to complete certain types of service, and a maximum number of service appointments per day.

### Problem Details:
- 8 Locations each with an associated demand
    - 1 Headquarters 
        - service type 1 demand: [0]
        - service type 2 demand: [1]
        - headquarters hours of operation: [5,20]
    - 7 Service Locations
        - service type 1 demand: [1, 1, 1, 0, 0, 0, 0]
        - service type 2 demand: [0, 0, 1, 1, 1, 1, 1]
        - service locations time windows: [[9,12],[9,12],[11,14],[13,16],[13,16],[13,16],[13,16]]
        - service location service times: [ 1, 1, 1.5, 0.5, 0.5, 0.5]

- 3 Delivery vehicles each with an associated capacity
    - 3 service technicians
        - capacity for service type 1: [2, 1, 0]
        - capacity for service type 2: [0, 1, 4]
        - technician availability [[9,17], [12,15], [9,17]]
        

Below we visualize the service locations with respect to the service company headquarters. The cost from all locations to all other locations (a cost matrix) will be required for optimization. To see an example of cost matrix generation from map data or a waypoint graph, refer to the [cost_matrix_creation.ipynb](cost_matrix_creation.ipynb) notebook.  For the purpose of this simple example we will omit the cost matrix calculation.

In [ ]:
location_names       = [ "Headquarters",     "A",    "B",    "C",    "D",    "E",    "F",    "G"  ]
location_coordinates = [     [4, 4],        [1, 3], [8, 1], [2, 1], [6, 7], [0, 2], [7, 6], [5, 3] ]
location_coordinates_df = pd.DataFrame(location_coordinates, columns=['xcord', 'ycord'], index=location_names)
utils.gen_plot(location_coordinates_df).show()

### Cost Matrix

The cost matrix dictates the cost of travel between locations of interest.  The cost itself can be anything relevant to the user.  In this case we are constraining time window constraints.  When constraining time windows for locations or vehicles it is assumed (if only a single cost matrix is provided) that it represents time. 

Here is the cost(time) matrix corresponding to the locations above:

In [ ]:
time_matrix = [
 [0.00, 0.31, 0.50, 0.36, 0.36, 0.44, 0.36, 0.14],
 [0.31, 0.00, 0.72, 0.22, 0.64, 0.14, 0.67, 0.40],
 [0.50, 0.72, 0.00, 0.60, 0.63, 0.80, 0.51, 0.36],
 [0.36, 0.22, 0.60, 0.00, 0.72, 0.22, 0.70, 0.36],
 [0.36, 0.64, 0.63, 0.72, 0.00, 0.77, 0.14, 0.41],
 [0.44, 0.14, 0.80, 0.22, 0.77, 0.00, 0.80, 0.51],
 [0.36, 0.67, 0.51, 0.70, 0.14, 0.80, 0.00, 0.36],
 [0.14, 0.40, 0.36, 0.36, 0.41, 0.51, 0.36, 0.00]
]

# Create a dataframe of this matrix
time_matrix_df  = pd.DataFrame(time_matrix, 
                              index=location_coordinates_df.index, 
                              columns=location_coordinates_df.index)
time_matrix_df

### Service Locations

Setup the service location data

In [ ]:
# exclude head quarters from service location names
service_location_ids = [1, 2, 3, 4, 5, 6, 7]
service_location_names = [location_names[i] for i in service_location_ids]
service_location_data = {
    "service_location_names": service_location_names,
    "service_location_ids": service_location_ids,
    "service_type1_demand": [1, 1, 1, 0, 0, 0, 0],
    "service_type2_demand": [0, 0, 1, 1, 1, 1, 1],
    "location_earliest_time": [9, 9, 11, 13, 13, 13, 13],
    "location_latest_time": [12, 12, 14, 16, 16, 16,16],
    "required_service_time": [1, 1, 1.5, 0.5, 0.5, 0.5, 0.5]
}
service_location_data_df = pd.DataFrame(service_location_data).set_index('service_location_names')
service_location_data_df

### Vehicles

Setup vehicle/technician data

In [ ]:
n_vehicles = 3
vehicle_data = {
    "vehicle_ids": [i for i in range(n_vehicles)],
    "capacity_service_type1":[2, 1, 0],
    "capacity_service_type2":[0, 1, 4],
    "vehicle_availability_earliest":[9, 11, 9],
    "vehicle_availability_latest":[17, 15, 17]
}
vehicle_data_df = pd.DataFrame(vehicle_data).set_index('vehicle_ids')
vehicle_data_df

# Initialize cuOpt Service Client and cuOpt Problem Data

In [ ]:
cuopt_client_id = os.environ["CUOPT_CLIENT_ID"]
cuopt_client_secret = os.environ["CUOPT_CLIENT_SECRET"]


cuopt_service_client = CuOptServiceClient(
    client_id=cuopt_client_id,
    client_secret=cuopt_client_secret,
    )

cuopt_problem_data = {}

### Set Cost Matrix

In [ ]:
cuopt_problem_data["cost_matrix_data"] = {
        "cost_matrix": {
            "0": time_matrix
        }
    }

### Set Fleet Data

In [ ]:
cuopt_problem_data["fleet_data"] = {
        "vehicle_locations": [[0,0]] * n_vehicles,
        "capacities": [vehicle_data["capacity_service_type1"], vehicle_data["capacity_service_type2"]],
        "vehicle_time_windows": list(zip(vehicle_data['vehicle_availability_earliest'],
                                         vehicle_data['vehicle_availability_latest']))
}

### Set Task Data

In [ ]:
cuopt_problem_data["task_data"] = {
        "task_locations": service_location_ids,
        "demand": [service_location_data["service_type1_demand"], 
                   service_location_data["service_type2_demand"]],
        "task_time_windows": list(zip(service_location_data["location_earliest_time"],
                                          service_location_data["location_latest_time"])),
        "service_times": service_location_data["required_service_time"]
}

### Set Solver Configuration

In [ ]:
cuopt_problem_data["solver_config"] = {
        "time_limit": 5
    }

### Get Optimized Routes

In [ ]:
# Solve the problem
solver_response = cuopt_service_client.get_optimized_routes(
    cuopt_problem_data
)

# Process returned data
solver_resp = solver_response["response"]["solver_response"]

if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    utils.show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

**Notice** that this solution leverages the fact that vehicle 1 is the only vehicle with the ability to perform both service type 1 and service type 2.  In addition, vehicle 0 and vehicle 2 also serve the locations they are suited to service and minimize the time taken along these routes.

In [ ]:
vehicle_colors = ["red", "green", "blue"]
utils.map_vehicle_routes(location_coordinates_df, solver_resp, vehicle_colors).show()